In [43]:
file = spark.sparkContext.textFile("file:///C:/Users/Ashwini/Desktop/onTimeAirDepart")

In [44]:
file.count()

8088

In [45]:
file.take(5)

['ABE ALB\t10.0',
 'ABE ATL\t9.94767144319345',
 'ABE AVP\t3.4390070921985814',
 'ABE BDL\t0.0',
 'ABE BHM\t11.0']

In [52]:
file.filter(lambda row :"ABI" in row).take(20)

['ABI DFW\t3.8689159922489456',
 'ABI IAH\t5.570802005012531',
 'ABI LAX\t0.0',
 'AEX ABI\t0.0',
 'DFW ABI\t10.273330685879616',
 'IAH ABI\t6.560549313358302',
 'IND ABI\t-2.0',
 'OAK ABI\t-180.0',
 'ONT ABI\t108.0']

In [46]:
file1 = file.map(lambda line:line.split()).filter(lambda line:len(line)==3)
file1.take(10)

[['ABE', 'ALB', '10.0'],
 ['ABE', 'ATL', '9.94767144319345'],
 ['ABE', 'AVP', '3.4390070921985814'],
 ['ABE', 'BDL', '0.0'],
 ['ABE', 'BHM', '11.0'],
 ['ABE', 'BWI', '3.2491816693944355'],
 ['ABE', 'CLE', '-3.8152038454691115'],
 ['ABE', 'CLT', '1.228503421600714'],
 ['ABE', 'CVG', '3.195371205290051'],
 ['ABE', 'DCA', '0.3835978835978836']]

In [47]:
file.count()

8088

In [12]:
from pyspark.sql import Row
rdd = file.map(lambda row: Row(src_airport=row[0],dest_airport=row[1], dep_delay=row[2]))

In [28]:
df = spark.createDataFrame(rdd)

In [34]:
from pyspark.sql.functions import ceil, col,round

df1 = df.select("src_airport","dest_airport",round(col('dep_delay'),4))

In [37]:
df1 =df1.withColumnRenamed("round(dep_delay, 4)","dep_delay")

In [35]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)

In [38]:
df1.write\
.format("org.apache.spark.sql.cassandra")\
.mode('append')\
.options(table = "airport_depart_delay", keyspace = "aviation")  \
.save()

In [21]:
data= sqlContext.read \
  .format('org.apache.spark.sql.cassandra') \
  .options(table='airport_depart_delay', keyspace='aviation') \
  .load()

In [22]:
data.show()

+-----------+--------------------+------------+
|src_airport|           dep_delay|dest_airport|
+-----------+--------------------+------------+
|        LYH|13.44827586206896...|         BWI|
|        LYH|8.638381201044387000|         CHO|
|        LYH|6.861136826513609000|         ATL|
|        LYH|3.226510067114094000|         CLT|
|        LYH|0.250000000000000000|         ROA|
|        CRW|106.0000000000000...|         ROA|
|        CRW|100.0000000000000...|         RDU|
|        CRW|14.90111991259218...|         ORD|
|        CRW|12.77566784679755...|         ATL|
|        CRW|6.059131736526946000|         DCA|
|        CRW|5.608695652173913000|         HTS|
|        CRW|5.521376491383120000|         PIT|
|        CRW|5.068027210884353000|         LEX|
|        CRW|5.064402810304450000|         LGA|
|        CRW|5.000000000000000000|         BNA|
|        CRW|4.678974132213133000|         CLT|
|        CRW|4.677583465818760000|         IAH|
|        CRW|3.134252169474221400|      

In [48]:
file.filter(lambda row:row[0]=='CMI').take(7)

[]